In [83]:
import numpy as np
import pandas as pd
import xarray as xr
import numba as nb
import dask

import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, log_loss
from sklearn.model_selection import ParameterSampler
from sklearn.utils import shuffle
import sklearn

from xgboost import XGBClassifier

from glob import glob
from tqdm import tqdm
import itertools 
import time
import os
import copy

from typing import Optional, List, Union
import logging
from IPython.utils import io

import spect

In [84]:
import imp
imp.reload(spect)

<module 'spect' from '/exports/csce/datastore/geos/users/s1205782/Projects/spect/notebooks/spect.py'>

# **Load and proprocess data**

In [85]:
BINS = np.arange(0, 1350, 50)
MZ_THRESH = 100

In [ ]:
ds_bin = spect.get_binned_data(bins=BINS, mz_thresh=MZ_THRESH)

In [ ]:
ds_bin

In [11]:
ds_scale = ds_bin['abundance'].mean(dim=('temp_bins', 'mz')).groupby(ds_bin.instrument_type).mean()
missing_fill = np.nan

def preprocess(ds, drop_he=False, rebase=False, maxscale=True, scale_int=True, log_scale=False, nan_fill=missing_fill):
    ds_pro = ds.copy(deep=True)
    ds_pro['features'] = ds_pro['abundance'].clip(0, None)
    ds_pro['integrated_abundance'] = ds_pro['features'].mean(dim='temp_bins')
    if scale_int:
        ds_pro['integrated_abundance'] = (
            (ds_pro['integrated_abundance'].groupby(ds.instrument_type)/ds_scale)
            .drop('instrument_type')
        )
    if rebase:
        ds_pro['features'] = ds_pro.features - ds_pro.features.min(dim=('temp_bins'))
    if drop_he:
        ds_pro = ds_pro.drop_sel(mz=4)
    if maxscale:
        ds_pro['features'] = ds_pro.features/ds_pro.features.max(dim=('temp_bins', 'mz'))
    if log_scale:
        ds_pro['features'] = np.log10(ds_pro['features'].clip(1e-4,None))+3
        ds_pro['integrated_abundance'] = np.log10(ds_pro['integrated_abundance'].clip(1e-4,None))+3
    ds_pro = ds_pro.fillna(nan_fill)
    return ds_pro

## Process data

In [12]:
from skmultilearn.model_selection import iterative_train_test_split

def base2_split(indices, y, splits):
    log2_splits = np.log2(splits)
    assert log2_splits==int(log2_splits)
    if log2_splits==1:
        indices_1, _, indices_2, _ = iterative_train_test_split(indices, y, test_size=0.5)
        return [indices_1, indices_2]
    else:
        indices_1, y_1, indices_2, y_2 = iterative_train_test_split(indices, y, test_size=0.5)
        return base2_split(indices_1, y_1, splits//2)+base2_split(indices_2, y_2, splits//2)

def cross_val_splits(ds, splits, random_seed=None):
    np.random.seed(random_seed)
    y = ds.labels.values
    indices = shuffle(ds.sample_id.values[:,None])
    partitions = base2_split(indices, y, splits=splits)
    ds_train_test = []
    for p in partitions:
        ds_train_test.append((ds.drop_sel(sample_id=p[:,0]), ds.sel(sample_id=p[:,0])))
    return ds_train_test

### Final dataset

In [13]:
ds_final = ds_bin.where(ds_bin.split=='train', drop=True)

ds_final

<xarray.Dataset>
Dimensions:          (sample_id: 766, temp_bins: 26, mz: 101, species: 10)
Coordinates:
  * sample_id        (sample_id) object 'S0000' 'S0001' ... 'S0764' 'S0765'
  * mz               (mz) int64 0 1 2 3 4 5 6 7 8 ... 93 94 95 96 97 98 99 100
  * temp_bins        (temp_bins) object (0, 50] (50, 100] ... (1250, 1300]
  * species          (species) <U14 'basalt' 'carbonate' ... 'sulfate' 'sulfide'
Data variables:
    abundance        (sample_id, temp_bins, mz) float64 5.543e-11 ... nan
    time             (sample_id, temp_bins) float64 61.56 266.8 ... nan nan
    temp             (sample_id, temp_bins) float64 40.9 74.9 122.9 ... nan nan
    labels           (sample_id, species) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    split            (sample_id) object 'train' 'train' ... 'train' 'train'
    instrument_type  (sample_id) object 'commercial' ... 'sam_testbed'

In [14]:
def this_preprocess(ds):
    return preprocess(ds, drop_he=False, rebase=False, maxscale=True, scale_int=True, log_scale=False)

# **Model**

## Functions

In [22]:
np.random.normal([[-1, 5], [1,1]], scale=[.01,.30])

array([[-0.98558701,  5.19620233],
       [ 0.98952383,  1.23170527]])

In [78]:
default_xgboost_params = dict(
    est_patience=10,
    model_params = dict(
        min_child_weight = 2,
        gamma = .1,
        subsample = 0.9,
        colsample_bytree = 0.9,
        reg_alpha = 0.01,
        max_depth = 6,
        learning_rate = 0.05,
        n_estimators = 500,
        max_delta_step = 1,
        scale_pos_weight = 8,
    ),
)


class XG:
    def __init__(self,
            xgboost_params=default_xgboost_params,
            ):
        
        self._single_xgboost = XGBClassifier(
            n_jobs=20,
            eval_metric='logloss',
            gpu_id=0,
            use_label_encoder=False,
            missing=missing_fill,
            tree_method='exact',
            **xgboost_params['model_params']
        )
        
        self.xgboost_params = xgboost_params
        
        self.xgboost = []
        self.logloss_report = None
    
    def _prepare_data_xgboost(self, ds, get_y=True):
        X = [
            ds.features.stack(dict(z=("temp_bins","mz"))).values, 
            (ds.instrument_type.values=='commercial').astype(float)[:, None],
            ds.integrated_abundance.values,
            ds.time.values,
        ]
        X = np.concatenate(X, axis=1)
        
        if get_y:
            y = ds.labels.values
            return X, y
        else:
            return X
        
    
    def _fit_xgboost(self, ds_train, ds_val):
        
        X_train, y_train_all = self._prepare_data_xgboost(ds_train)
        X_val, y_val_all = self._prepare_data_xgboost(ds_val)
        
        self.stds = np.nanstd(X_train, axis=0)
        
        for col in tqdm(range(y_train_all.shape[1]), desc='Fitting xgboost'):
            y_train = y_train_all[:, col]
            y_val = y_val_all[:, col]

            this_clf = sklearn.base.clone(self._single_xgboost)
            
            this_clf.fit(
                X_train, 
                y_train,
                eval_set=((X_train, y_train), (X_val, y_val),),
                early_stopping_rounds=self.xgboost_params['est_patience'], ##### TODO
                eval_metric="logloss",
                verbose=0
                          
            )
            
            self.xgboost.append(this_clf)

        return
    
    def fit(self, ds_train, ds_val):
        # fit the net
        self.species = ds_train.species.values
        self._fit_xgboost(ds_train, ds_val)
        self.logloss_report = self.estimate_logloss(ds_train, ds_val)
    
    def _predict_proba(self, ds, tta=False, tta_scale=.1):
        X = self._prepare_data_xgboost(ds, get_y=False)
        
        y_preds = []
        for col in range(self.species.shape[0]):
            yp = self.xgboost[col].predict_proba(X)[:, 1:]
            for _ in range(tta):
                X_samp = X*np.random.normal(1, scale=tta_scale, size=X.shape)
                yp += self.xgboost[col].predict_proba(X_samp)[:, 1:]
            y_preds += [yp/(tta+1)]
        return np.concatenate(y_preds, axis=1)
    
    def predict_proba(self, ds, tta=False, tta_scale=.01):
        y_preds = self._predict_proba(ds, tta=tta, tta_scale=tta_scale)
        ds_pred = xr.DataArray(
            y_preds, 
            dims=['sample_id', 'species'], 
            coords=[ds.sample_id.values, self.species], 
            name='preds'
        )
        return ds_pred
    
    def estimate_logloss(self, ds_train, ds_val, tta=10, tta_scale=.1):
        boosted_train = xr_loss(self.predict_proba(ds_train), ds_train.labels)
        boosted_val = xr_loss(self.predict_proba(ds_val), ds_val.labels)
        boosted_val_tta = xr_loss(self.predict_proba(ds_val,  tta=tta, tta_scale=tta_scale), ds_val.labels)
        
        losses = xr.merge([
            boosted_train.to_dataset(name='boosted_train'),
            boosted_val.to_dataset(name='boosted_val'),
            boosted_val_tta.to_dataset(name='boosted_val_tta'),
        ])
        return losses
    
def xr_loss(ds_pred, ds_target):
    return -(ds_target*np.log(ds_pred)+(1-ds_target)*np.log(1-ds_pred)).mean(dim=('sample_id'))

In [67]:
ds_train_test_splits = cross_val_splits(this_preprocess(ds_final), 8, random_seed=317984)

In [68]:
def tprint(s):
    print(f"{time.asctime()} - {s}")

In [82]:
boosted.estimate_logloss(*split, tta=100, tta_scale=.100).mean(dim='species')

<xarray.Dataset>
Dimensions:          ()
Data variables:
    boosted_train    float64 0.02528
    boosted_val      float64 0.1575
    boosted_val_tta  float64 0.1677

In [79]:
cross_val_results = []
for i, split in enumerate(ds_train_test_splits):
    #if i==0: continue
    tprint(f'Split {i+1} of {len(ds_train_test_splits)}')
    params = copy.deepcopy(default_xgboost_params)
    #params['model_params']['n_estimators']=10
    boosted = XG(xgboost_params=params)
    boosted.fit(*split)
    print(boosted.logloss_report.mean(dim='species'))
    
    cross_val_results.append(boosted)
    break

Thu Mar 17 16:02:43 2022 - Split 1 of 8


Fitting xgboost: 100%|██████████████████████████████████████████████████████████████████| 10/10 [06:54<00:00, 41.46s/it]


<xarray.Dataset>
Dimensions:          ()
Data variables:
    boosted_train    float64 0.02528
    boosted_val      float64 0.1575
    boosted_val_tta  float64 0.1651
